## Importing Necessary Libraries and modules

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import cv2
import imghdr

## Creating Bath size and Image inputs Dimensions of our Dataset

In [ ]:
batch_size = 64
img_height = 48
img_width = 48

## Creating our Training and Validation Data

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
 "../input/fer2013/train",
  labels = "inferred",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale',
  label_mode="categorical")

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
 "../input/fer2013/test",
  labels = 'inferred',
  label_mode = "categorical",
  seed=123,
  image_size=(48, 48),
  batch_size=batch_size,
  color_mode='grayscale')

## Obtaining Classes or Labels of our Training and Validation Dataset

In [ ]:
class_names = train_ds.class_names
class_names

## Obtaining Image_batch and label_batch of our Datasets

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

### Normalizing the Pixel Values of our Datasets
This is done so as to regularize the pixel values of the images between 0 and 1.

In [ ]:
train_ds = train_ds.map(lambda x,y: (x/255, y))
val_ds = val_ds.map(lambda x,y: (x/255, y))

## Confirming our Regularized Datasets

In [ ]:
train_ds.as_numpy_iterator().next()[0].min()

In [ ]:
val_ds.as_numpy_iterator().next()[0].max()

## Configuring Dataset for a Better Performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Creating a Simple Neural Network
We will be creating the first neural network that will consist of 4 block of layers. This will act as a sort of control model. What this means is that we will be comparing it with the second model which will be a tuned one.

In [ ]:
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

model_1 = keras.Sequential([
    layers.Conv2D(64, (3,3), activation="relu", input_shape=(img_height, img_width, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, (3,3), activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, (3,3), activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
                  
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(7, activation="softmax")
    ])



## Compiling our Model


In [ ]:
model_1.compile("adam",
  loss="categorical_crossentropy",
    metrics=['accuracy'])


## Summary of our Model

In [ ]:
model_1.summary()

## Training our Model

In [ ]:
history_1 = model_1.fit(
  train_ds,
  validation_data=val_ds,
  epochs=20,
)


## Accuracy and Loss Curve

In [ ]:
import pandas as pd

history_frame_1 = pd.DataFrame(history_1.history)
history_frame_1.loc[:, ["accuracy", "val_accuracy"]].plot()
plt.title("Accuracy Curve_1")
history_frame_1.loc[:, ["loss", "val_loss"]].plot()
plt.title("Loss Curve_1")
plt.show()

## Creating a tuned Deep Neural Network
This neural network consist of 6 block of layers. We did some data augmentation by flipping our image horizontally and thereby creating more examples for our model to learn from and this was done using the `RandomFlip()` function. Also, we had to zoom into our image using the `RandomZoom()` function. The essence of data augmentation is simply to enhance the data passed to the model in such a way that more features from the data is provided to the model for learning.

Also, we ensure the `padding` parameter is `same` in each of the subsequent `Maoling2D` or `Conv2D` after their first occurence in each blocks when creating such a large neural network so as to avoid `ValueError`.

In [ ]:

model_2 = keras.Sequential([
    layers.RandomFlip("horizontal", input_shape =(img_height, img_width, 1)),
    layers.RandomZoom(0.2, 0.3),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2,2), padding='same'),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2,2), padding='same'),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2,2), padding='same'),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2,2), padding='same'),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3,3), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2,2), padding='same'),
    layers.Dropout(0.25),
                  
    layers.Flatten(),
    
    layers.Dense(256, activation='relu'),
    layers.Dense(7, activation='softmax'),
])

## Compiling our Model
We add a learning rate to your network when compiling. The learning rate controls how quickly or slowly a neural network model learns a problem.

In [ ]:
model_2.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, epsilon=1e-06),
  loss="categorical_crossentropy",
    metrics=['accuracy'])


## Summary of our Model

In [ ]:
model_2.summary()

## Training our Model

In [ ]:
history_2 = model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=120,
)


### Loss and Accuracy Curve

In [ ]:

history_frame_2 = pd.DataFrame(history_2.history)
history_frame_2.loc[:, ["accuracy", "val_accuracy"]].plot()
plt.title("Accuracy Curve_2")
history_frame_2.loc[:, ["loss", "val_loss"]].plot()
plt.title("Loss Curve_2")
plt.show()


## Observation
We can now see that the newly created model, `model-2`, is a better model in comparison to the first model. 

## Model Evaluation

We evaluate our trained model using our validation dataset. 

In [ ]:
score = model_2.evaluate(val_ds)
score = 100 * round(score[1], 3)
print(f"The accuracy of this model is {score}%")

We can see that the model did well on the validation data with an accuracy of 64.1%.

## Saving our Model

We need to save our model should in case we need to make use of it outside this environment.

In [ ]:
model_1.save("emotion_detector.h5")

now dowload the emotion_detector.h5 and go to your computer , use this code to test your model

copy the last cell and execute the code or go to github and download the repository

github: https://github.com/imenselmi/Emotion-Recognition-using-CNN.git

dataset drive:https://drive.google.com/drive/folders/1mFOMxCEPBW5vL5JsYf2UZSubGrVH1le8?usp=sharing

In [ ]:
import cv2
import numpy as np
from keras.models import model_from_json


emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# load json and create model
json_file = open('emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("emotion_model.h5")
print("Loaded model from disk")

# start the webcam feed
#cap = cv2.VideoCapture(0)

# pass here your video path
cap = cv2.VideoCapture("vid.mp4")

while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()